# Glass Melter Level Modeling with Neural ODEs

This notebook demonstrates the complete workflow:
1. Physics-based simulation
2. Neural ODE training (v1 — simulation data)
3. Evaluation with state feedback correction
4. Publication-quality visualization

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))

import torch
import numpy as np
import matplotlib.pyplot as plt

from glass_melter_level.config import GlassMelterParams
from glass_melter_level.models.physics import GlassMelterODE, simulate_scenario
from glass_melter_level.models.neural_ode import (
    NeuralODE_GlassMelter,
    integrate_neural_ode,
    integrate_neural_ode_with_correction,
    save_model,
    load_model,
)
from glass_melter_level.data.scenario_generation import generate_scenarios
from glass_melter_level.training.trajectory import prepare_trajectory
from glass_melter_level.training.trainer import NeuralODETrainer
from glass_melter_level.evaluation.metrics import compute_metrics, print_metrics_row
from glass_melter_level.visualization.style import setup_publication_style, get_pub_colors, add_subplot_label
from glass_melter_level.visualization.plots import plot_state_comparison, plot_model_comparison_bars

setup_publication_style()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

## 1. Physics-Based Simulation

Generate a single scenario using the first-principles ODE model.

In [ ]:
params = GlassMelterParams()
print(params)

# Simulate a ramp-up scenario
t_sim = 48.0  # hours
dt = 0.01
t = np.arange(0, t_sim, dt)

u1 = 8.0 * np.ones_like(t)  # Constant charging
u2 = 5.0 + 1.0 * np.sin(2 * np.pi * t / 24)  # Daily pull cycle
w = np.zeros_like(t)

sim = simulate_scenario(params, t, u1, u2, w)

fig, axes = plt.subplots(3, 1, figsize=(10, 8), sharex=True)
axes[0].plot(sim['t'], sim['h']); axes[0].set_ylabel('h [m]')
axes[1].plot(sim['t'], sim['v']); axes[1].set_ylabel('v [m/h]')
axes[2].plot(sim['t'], sim['q_m']); axes[2].set_ylabel('q_m [m³/h]')
axes[2].set_xlabel('Time [h]')
plt.tight_layout()
plt.show()

## 2. Generate Training Scenarios & Train Neural ODE v1

In [ ]:
# Generate diverse scenarios
scenarios = generate_scenarios(
    n_scenarios=100,
    t_sim=48.0, dt=0.01,
    u1_range=(3.0, 12.0),
    u2_range=(2.0, 8.0),
    seed=42,
)
print(f'Generated {len(scenarios)} scenarios')

# Prepare trajectories
trajectories = [prepare_trajectory(s, subsample=10) for s in scenarios]
n_train = int(0.8 * len(trajectories))
train_traj = trajectories[:n_train]
val_traj = trajectories[n_train:]
print(f'Train: {len(train_traj)}, Val: {len(val_traj)}')

In [ ]:
# Create and train Neural ODE
model = NeuralODE_GlassMelter(params=params)
trainer = NeuralODETrainer(model, lr=1e-3)

history = trainer.train(
    train_traj, val_traj,
    n_epochs=50,  # Increase to 200 for full training
    rollout_steps=20,
    print_every=10,
)

In [ ]:
# Training loss curves
fig, ax = plt.subplots(figsize=(8, 4))
ax.semilogy(history['train'], label='Train')
ax.semilogy(history['val'], label='Validation')
ax.set_xlabel('Epoch')
ax.set_ylabel('MSE Loss')
ax.legend()
plt.tight_layout()
plt.show()

## 3. Evaluate: Open-Loop vs State Feedback Corrected

In [ ]:
# Test on a held-out scenario
test_sim = scenarios[-1]
test_traj = prepare_trajectory(test_sim, subsample=10)

model.eval()
with torch.no_grad():
    # Open-loop prediction
    x_open = integrate_neural_ode(model, test_traj['x0'], test_traj['u'], test_traj['t'])
    
    # With state feedback correction
    h_meas = test_traj['x'][:, 0].numpy()
    x_corr = integrate_neural_ode_with_correction(
        model, test_traj['x0'], test_traj['u'], test_traj['t'],
        h_measured=h_meas, correction_gain=0.3,
    )

# Metrics
h_true = test_traj['x'][:, 0].numpy()
m_open = compute_metrics(h_true, x_open[:, 0].numpy())
m_corr = compute_metrics(h_true, x_corr[:, 0].numpy())
print_metrics_row('Open', m_open, ' m')
print_metrics_row('Corrected', m_corr, ' m')

# Build results dict for plotting
results = {
    't': test_traj['t'].numpy(),
    'h_true': h_true,
    'h_pred_orig': x_open[:, 0].numpy(),
    'h_pred': x_corr[:, 0].numpy(),
    'v_true': test_traj['x'][:, 1].numpy(),
    'v_pred_orig': x_open[:, 1].numpy(),
    'v_pred': x_corr[:, 1].numpy(),
    'qm_true': test_traj['x'][:, 2].numpy(),
    'qm_pred_orig': x_open[:, 2].numpy(),
    'qm_pred': x_corr[:, 2].numpy(),
    'rmse_h': m_corr['rmse'],
}
fig = plot_state_comparison(results)
plt.show()

## 4. Save / Load Model

In [ ]:
# Save
save_model(model, '../trained_models/neural_ode_v1_demo.pt',
           metadata={'variant': 'v1', 'epochs': 50})

# Load
model_loaded = load_model('../trained_models/neural_ode_v1_demo.pt', params=params)
print(f'Model loaded: {sum(p.numel() for p in model_loaded.parameters())} parameters')